In [1]:
import pandas as pd

from dataset.svw import SVW
from tqdm import tqdm
svw = SVW()

videos = svw._get_filtered_videos()

In [2]:
from preprocessing.video import Video
import cv2
import numpy as np

def get_video_as_array(v: Video, frame_ids):
    cap = cv2.VideoCapture(v.video_reshaped_path.as_posix())
    video = []
    frame_id = 1
    while (cap.isOpened()):
        ret, img = cap.read()
        if not ret:
            break
        if frame_id in frame_ids:
            video.append(img)
        frame_id +=1
    cap.release()
    return video


In [3]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input, decode_predictions
import pickle as pkl

mobilenet = tf.keras.applications.MobileNetV3Large(
    input_shape=None,
    alpha=1.0,
    minimalistic=False,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    classes=1000,
    pooling='avg',
    dropout_rate=0.2,
    include_preprocessing=True,
)

def get_roi(frame):
    if frame.size == 0:
        return np.zeros((224, 224, 3))

    roi = cv2.resize(frame, (224, 224))
    return np.array(roi)

with open('/home/rafa/PycharmProjects/sdm2-ready-code/preprocessing/pca.pkl', 'rb') as file:
    pca = pkl.load(file)



2023-05-14 18:51:34.695206: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from preprocessing.dto.detection_dto import DetectionDto
from pathlib import Path
from time import time
from tqdm import tqdm
progress = tqdm(videos)
skipped = 0

for vid_path in progress:
    vid = Video(vid_path)
    try:
        df = vid.build_detections()
    except:
        continue

    frames = df[DetectionDto.FRAME_ID].unique()
    progress.set_postfix({'path':vid.video_path.as_posix(), 'detections_num': frames.shape, 'skipped': skipped})
    save_path = Video._add_suffix_to_path(vid_path, '_detections_big_ext.pkl')
    if save_path.exists() :
        continue

    if df.empty:
        skipped += 1
        with open('/home/rafa/SVW/skipped1.txt', 'a') as file:
            file.write(f'{vid_path.as_posix()} \t - empty \n')
        continue

    if  df.shape[0] > 2300:
        skipped += 1
        with open('/home/rafa/SVW/skipped1.txt', 'a') as file:
            file.write(f'{vid_path.as_posix()} \t - long \n')
        continue

    start_time = time()
    vid_as_arr = get_video_as_array(vid, frames)

    resized = np.array([get_roi(single_frame) for single_frame in vid_as_arr])

    preds = mobilenet.predict(resized, verbose=False)
    preds_red = pca.transform(preds)
    preds_list = [val for val in preds]
    preds_red_list = [val for val in preds_red]
    preds_df = pd.DataFrame({'right_frame_id': frames, 'frame_vis_features': preds_list, 'frame_vis_features_pca': preds_red_list})
    ext_df = df.merge(preds_df, 'inner', left_on='frame_id', right_on='right_frame_id')
    ext_df = ext_df.drop('right_frame_id', axis=1)
    ext_df.to_pickle(save_path)




100%|██████████| 3933/3933 [57:39<00:00,  1.14it/s, path=/home/rafa/SVW/Videos/baseball/2423___9845e29ab87745b082bfc50800996641.mp4, detections_num=(142,), skipped=0]         
